In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install geopandas

In [ ]:
!pip install typing_extensions==4.5.0 --upgrade
!pip install openai
!pip install tiktoken
!pip install --upgrade transformers
!pip install langchain
!pip install -U langchain-openai
!pip install torch

  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
fastapi 0.109.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
openai 1.10.0 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
  Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolve

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/dataset/madison/'

## Load dataset

In [ ]:
objects = pd.read_csv(path + '/input/objects_embeddings.csv')
objects[['embedding_ada']] = objects[['embedding_ada']].applymap(lambda s: np.array(eval(s)))
objects[['embedding_large']] = objects[['embedding_large']].applymap(lambda s: np.array(eval(s)))
objects.head()

,objectid,geometry,source,geom_type,n_tokens,embedding_ada,embedding_large
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",link,LineString,21,"[-0.014463399536907673, -0.004633771721273661,...","[0.01921754889190197, 0.01929466612637043, -0...."
1,"road_(53591318, 1179412798)","LINESTRING (-89.477 43.0693, -89.477 43.0681)",link,LineString,22,"[-0.014376881532371044, -0.004993679467588663,...","[0.021175693720579147, 0.015708766877651215, -..."
2,"road_(1178722373, 1178722211)","LINESTRING (-89.4626 43.0553, -89.4627 43.0554)",link,LineString,24,"[-0.015243860892951488, -0.004274553153663874,...","[0.011615517549216747, 0.02089725434780121, -0..."
3,"road_(53430628, 53468282)","LINESTRING (-89.3045 43.0246, -89.3053 43.0252)",link,LineString,24,"[-0.016633406281471252, -0.0014020365197211504...","[0.0076748463325202465, 0.030930323526263237, ..."
4,"road_(3444117858, 6680557104)","LINESTRING (-89.5081 43.0066, -89.5072 43.006)",link,LineString,23,"[-0.010978729464113712, -0.005506434012204409,...","[0.03258766606450081, 0.024168409407138824, -0..."


In [ ]:

relations = pd.read_csv(path+'input/relations.csv', usecols=['subjectid', 'objectid', 'predicate', 'relate', 'idx'])
# relations.to_csv('/content/drive/MyDrive/dataset/madison/output/relations_update.csv', index=False)
# relations = pd.read_csv('/content/drive/MyDrive/dataset/madison/output/relations_update.csv')
relations.head()

,objectid,subjectid,predicate,relate,idx
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0
1,"road_(2634716954, 8997363243)","road_(8997363242, 2634716954)",contains,101F00FF2,1
2,poi_way/1134399612,poi_way/1134399613,contains,1F1FFFFF2,2
3,"road_(1998162788, 1998162839)",poi_way/257174438,contains,101FFFFF2,3
4,"road_(3420767254, 3590603510)",poi_way/287637061,contains,101FFFFF2,4


In [ ]:
relations = pd.merge(relations, objects[['objectid', 'geometry', 'source', 'geom_type','n_tokens']].rename(columns={'objectid': 'subjectid',
                                                        'geometry': 'geometry_subject',
                                                        'source': 'source_subject',
                                                        'geom_type': 'geom_type_subject',
                                                          'n_tokens': 'n_tokens_subject'
                                                        }), on='subjectid', how='inner')
relations = pd.merge(relations, objects[['objectid', 'geometry', 'source', 'geom_type', 'n_tokens']], on='objectid', how='inner')
relations.head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24
1,"road_(53496379, 5657142535)","road_(53496379, 5656899646)",contains,101F00FF2,31,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24
2,"road_(5657142535, 53496379)","road_(5656899646, 53496379)",contains,101F00FF2,202,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24
3,"road_(5657142535, 53496379)","road_(53496379, 5656899646)",contains,101F00FF2,54,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24
4,"road_(5657142535, 53496379)",census_550250020003,contains,102FF1FF2,180,"POLYGON ((-89.3515 43.0928, -89.346 43.0939, -...",cbg,Polygon,109,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24


In [ ]:
relations[(relations['geom_type_subject']=='Point')&(relations['geom_type']=='LineString')].sort_values(by=['n_tokens_subject', 'n_tokens']).iloc[0]

objectid                            road_(2633805462, 3423901632)
subjectid                                         node_3752320724
predicate                                                disjoint
relate                                                  FF0FFF102
idx                                                           195
geometry_subject                           POINT (-89.456 43.061)
source_subject                                       intersection
geom_type_subject                                           Point
n_tokens_subject                                               10
geometry             LINESTRING (-89.459 43.0661, -89.459 43.066)
source                                                       link
geom_type                                              LineString
n_tokens                                                       21
Name: 1383, dtype: object

In [ ]:
relate_df = relations[['geom_type_subject', 'predicate', 'geom_type']].drop_duplicates().reset_index(drop=True)
relate_df['rid'] = relate_df.index
relate_df['output'] = '(' + relate_df['geom_type_subject'] + ',' + relate_df['predicate'] + ',' + relate_df['geom_type'] + ')'
relations = pd.merge(relations, relate_df, on=['geom_type_subject', 'predicate', 'geom_type'], how='left')
# relations = pd.merge(relations, relate_df, on=['geom_type_subject', 'predicate', 'geom_type'], how='left')

In [ ]:
relations[['objectid', 'geom_type', 'geom_type_subject', 'predicate']].groupby(['geom_type_subject', 'predicate', 'geom_type']).count()

objectid
geom_type_subject predicate geom_type           
LineString        contains  LineString       225
                            Point            225
                  crosses   LineString       225
                            Polygon          225
                  disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    LineString       225
                  overlaps  LineString       225
                  touches   LineString       225
                            Point            225
                            Polygon          225
                  within    LineString       225
                            Polygon          225
Point             disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    Point            225
                  touches   LineString       225
                            Polygon          225
                  within    LineString       225
                            Polygon          225
Polygon           contains  LineString       225
                            Point            225
                            Polygon          225
                  crosses   LineString       225
                  disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    Polygon          225
                  overlaps  Polygon          225
                  touches   LineString       225
                            Point            225
                            Polygon          225
                  within    Polygon          225

In [ ]:
relations[['objectid', 'predicate']].groupby(['predicate']).count()

,objectid
predicate,
contains,1125
crosses,675
disjoint,2025
equals,675
overlaps,450
touches,1800
within,1125


In [ ]:
intersections = pd.read_csv(path + 'input/intersections.csv')
intersections.head()

,objectid,geometry,subjectid,geometry_subject,geom_type_subject,geom_type,predicate,pid,relate
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)","road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",LineString,LineString,equals,0,1FFF0FFF2
1,node_3455066029,POINT (-89.4513 43.0502),node_3455066029,POINT (-89.4513 43.0502),Point,Point,equals,0,0FFFFFFF2
2,node_3455066034,POINT (-89.451 43.0502),node_3455066034,POINT (-89.451 43.0502),Point,Point,equals,0,0FFFFFFF2
3,"road_(53509392, 53412067)","LINESTRING (-89.451 43.0476, -89.451 43.0484)","road_(53509392, 53412067)","LINESTRING (-89.451 43.0476, -89.451 43.0484)",LineString,LineString,equals,0,1FFF0FFF2
4,node_1142903570,POINT (-89.3634 43.0935),node_1142903570,POINT (-89.3634 43.0935),Point,Point,equals,0,0FFFFFFF2


## Train / Test split

In [ ]:
relations_example = relations[relations['idx']>=200].reset_index(drop=True)

relations_test = relations[relations['idx']<40].reset_index(drop=True)

relations_train = relations[(relations['idx']>=40)&(relations['idx']<200)].reset_index(drop=True)

In [ ]:
relations.head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,output
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
1,"road_(53496379, 5657142535)","road_(53496379, 5656899646)",contains,101F00FF2,31,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
2,"road_(5657142535, 53496379)","road_(5656899646, 53496379)",contains,101F00FF2,202,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
3,"road_(5657142535, 53496379)","road_(53496379, 5656899646)",contains,101F00FF2,54,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
4,"road_(5657142535, 53496379)",census_550250020003,contains,102FF1FF2,180,"POLYGON ((-89.3515 43.0928, -89.346 43.0939, -...",cbg,Polygon,109,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,1,"(Polygon,contains,LineString)"


In [ ]:
relations_test.sort_values(by=['rid', 'idx']).head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,output
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
2,"road_(2634716954, 8997363243)","road_(8997363242, 2634716954)",contains,101F00FF2,1,"LINESTRING (-89.3554 43.1351, -89.3554 43.1357)",link,LineString,24,"LINESTRING (-89.3554 43.1357, -89.3554 43.1355)",link,LineString,24,0,"(LineString,contains,LineString)"
3,poi_way/1134399612,poi_way/1134399613,contains,1F1FFFFF2,2,"LINESTRING (-89.4524 43.0716, -89.4521 43.0716...",poi,LineString,57,"LINESTRING (-89.4524 43.0715, -89.4521 43.0715...",poi,LineString,46,0,"(LineString,contains,LineString)"
6,"road_(1998162788, 1998162839)",poi_way/257174438,contains,101FFFFF2,3,"LINESTRING (-89.3392 43.0637, -89.3399 43.0641...",poi,LineString,134,"LINESTRING (-89.338 43.0659, -89.3378 43.0659)",link,LineString,23,0,"(LineString,contains,LineString)"
10,"road_(3420767254, 3590603510)",poi_way/287637061,contains,101FFFFF2,4,"LINESTRING (-89.5168 43.0606, -89.5168 43.0616...",poi,LineString,57,"LINESTRING (-89.5175 43.0611, -89.5175 43.0607)",link,LineString,24,0,"(LineString,contains,LineString)"


In [ ]:
relations_test[['objectid', 'geom_type', 'geom_type_subject', 'predicate']].groupby(['geom_type_subject', 'predicate', 'geom_type']).count()

objectid
geom_type_subject predicate geom_type           
LineString        contains  LineString        40
                            Point             40
                  crosses   LineString        40
                            Polygon           40
                  disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    LineString        40
                  overlaps  LineString        40
                  touches   LineString        40
                            Point             40
                            Polygon           40
                  within    LineString        40
                            Polygon           40
Point             disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    Point             40
                  touches   LineString        40
                            Polygon           40
                  within    LineString        40
                            Polygon           40
Polygon           contains  LineString        40
                            Point             40
                            Polygon           40
                  crosses   LineString        40
                  disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    Polygon           40
                  overlaps  Polygon           40
                  touches   LineString        40
                            Point             40
                            Polygon           40
                  within    Polygon           40

In [ ]:
data = relations_test[relations_test['predicate']!='disjoint'].reset_index(drop=True)

## Authentication

In [ ]:
# import tiktoken, os, openai

from google.colab import userdata

KEY = userdata.get('OPENAI_QSR')
ORG = userdata.get('OPENAI_ORG')
# openai.organization = ORG
# openai.api_key = KEY
# openai.Model.list()

In [ ]:
model = 'gpt-4'
llm_abbr = 'gpt4'
model_abbr = 'gpt4'

In [ ]:
from langchain_openai import ChatOpenAI
# process.env.OPENAI_ORGANIZATION = ORG


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

### Create vectordb

In [ ]:
!pip install chromadb

In [ ]:
embedding_model = "text-embedding-3-large" #"text-embedding-ada-002" #"text-embedding-3-large" #"text-embedding-ada-002" #"text-embedding-3-large" #"text-embedding-ada-002"
embed_abbr = "large" #"ada" #"large" #"ada"

In [ ]:
objects['vector_id'] = np.arange(objects.shape[0]).astype(str)
objects['embedding_%s'%embed_abbr] = objects['embedding_%s'%embed_abbr].apply(lambda s: s.tolist())

In [ ]:
# create collections
from langchain.embeddings.openai import OpenAIEmbeddings
# from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import chromadb

chroma_client = chromadb.EphemeralClient()

embedding_function = OpenAIEmbeddings(api_key=KEY, organization=ORG, model=embedding_model)

try:
    chroma_client.get_collection('objects')
    chroma_client.delete_collection('objects')
except:
    pass
objects_collection = chroma_client.create_collection(name='objects',
                                                     metadata={"hnsw:space": "cosine"})

# Add the content vectors
objects_collection.add(
    ids=objects.vector_id.tolist(),
    embeddings=objects['embedding_%s'%embed_abbr].tolist(),
    documents=objects.geometry.tolist(),
    metadatas=objects[['objectid']].to_dict('records')
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
from langchain_community.vectorstores import Chroma

db = Chroma(
    client=chroma_client,
    collection_name="objects",
    embedding_function=embedding_function,
)


In [ ]:
data = relations_test[relations_test['predicate']!='disjoint'].reset_index(drop=True)

In [ ]:
include_type='t'
entity = 'subject'

In [ ]:
llm = ChatOpenAI(model_name=model, temperature=0.7, api_key=KEY, organization=ORG)

## Direct query

### Without subject type

In [ ]:
import time
# llm_abbr = 'none'
qe_mode = 'none'
include_type = 'f'
entity = 'subject'
k_val = [5, 10, 20]

ranks = []
for idx, row in data.iloc[:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    question = 'Retrieve a geometry which ' + text


    subjects = db.similarity_search_with_score(question, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'subjectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['subjectid'][(intersections['objectid']==row['objectid'])&
                                            (intersections['predicate']==row['predicate'])]) - set([row['subjectid']])

    dists = dists[~(dists['subjectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['subjectid']==row['subjectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['subjectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/task2_%s_%s_%s.csv'%(qe_mode, include_type, entity), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

----------------
record 50 mrr: 0.12317580253856938, hits: [0.20930232558139536, 0.20930232558139536, 0.23255813953488372]
finish saving first 50 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, f


----------------
record 100 mrr: 0.09026049564078177, hits: [0.1797752808988764, 0.20224719101123595, 0.2696629213483146]
finish saving first 100 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, f


----------------
record 250 mrr: 0.054352649561625804, hits: [0.10628019323671498, 0.13526570048309178, 0.1642512077294686]
finish saving first 250 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, f


----------------
record 350 mrr: 0.07222039194129314, hits: [0.15613382899628253, 0.1821561338289963, 0.21189591078066913]
finish saving first 350 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, f


----------------
record 450 mrr: 0.06727892770061714, hits: [0.14285714285714285, 0.16666666666666666, 0.19345238095238096]
finish saving first 450 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, f


...

----------------
record 1100 mrr: 0.07655678318984327, hits: [0.13081761006289308, 0.16226415094339622, 0.2]
finish saving first 1100 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, f


----------------
record 1150 mrr: 0.07861070885221713, hits: [0.1324582338902148, 0.16348448687350836, 0.20047732696897375]
finish saving first 1150 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, f


----------------
record 1200 mrr: 0.07472003522251985, hits: [0.1255656108597285, 0.15610859728506787, 0.19117647058823528]
finish saving first 1200 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, f


----------------
record 1250 mrr: 0.07123144241109633, hits: [0.11948331539289558, 0.1496232508073197, 0.18299246501614638]
finish saving first 1250 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, f


----------------
record 1400 mrr: 0.08127517255597728, hits: [0.13076923076923078, 0.1605769230769231, 0.19423076923076923]
finish saving first 1400 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, f


In [ ]:
res = data[['subjectid', 'objectid', 'predicate']].reset_index().copy()
res_search = pd.DataFrame(ranks).rename(columns={
    'rank': 'rank_%s_%s_%s_%s'%(embed_abbr, llm_abbr, qe_mode, include_type),
    'dist': 'dist_%s_%s_%s_%s'%(embed_abbr, llm_abbr, qe_mode, include_type),
    'list': 'list_%s_%s_%s_%s'%(embed_abbr, llm_abbr, qe_mode, include_type),
})
res = pd.concat([res, res_search], axis=1)


res.to_csv(path+'result/task2/task2_%s_%s_%s_%s.csv'%(embed_abbr, llm_abbr, qe_mode, include_type), index=False)
res.head()

,index,subjectid,objectid,predicate,idx,rank_large_gpt4_none_f,dist_large_gpt4_none_f,list_large_gpt4_none_f
0,0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",contains,0,39,0.272096,"[poi_way/374898542, poi_way/297756530, road_(5..."
1,1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",contains,1,2,0.264627,"[poi_way/374898542, road_(53496379, 5656899646..."
2,2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",contains,2,2,0.282309,"[line_poi_way/345753995, road_(8997363242, 263..."
3,3,poi_way/1134399613,poi_way/1134399612,contains,3,2,0.257904,"[poi_way/1134399600, poi_way/1134399613, poi_w..."
4,4,parcel_70920121012,poi_way/1134399612,contains,4,3792,0.352876,"[poi_way/1134399600, poi_way/1134399604, poi_w..."


In [ ]:
res.columns

Index(['index', 'subjectid', 'objectid', 'predicate', 'idx', 'rank', 'dist',
       'qe', 'list'],
      dtype='object')

In [ ]:
k_val = [5, 10, 20]

mrr = (1./res['rank_%s_%s_%s_%s'%(embed_abbr, llm_abbr, qe_mode, include_type)].values).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank_%s_%s_%s_%s'%(embed_abbr, llm_abbr, qe_mode, include_type)].values<=k).mean())

mrr, hits

(0.08127517255597728,
 [0.13076923076923078, 0.1605769230769231, 0.19423076923076923])

In [ ]:
import time
# llm_abbr = 'none'
qe_mode = 'none'
include_type = 'all'
entity = 'subject'
k_val = [5, 10, 20]

ranks = []
for idx, row in data.iloc[:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    question = 'Retrieve a Point/LineString/Polygon geometry which ' + text


    subjects = db.similarity_search_with_score(question, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'subjectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['subjectid'][(intersections['objectid']==row['objectid'])&
                                            (intersections['predicate']==row['predicate'])]) - set([row['subjectid']])

    dists = dists[~(dists['subjectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['subjectid']==row['subjectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['subjectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/task2_%s_%s_%s.csv'%(qe_mode, include_type, entity), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

----------------
record 50 mrr: 0.05864878514911486, hits: [0.14, 0.18, 0.2]
finish saving first 50 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, all


----------------
record 100 mrr: 0.0578571718106941, hits: [0.11, 0.15, 0.17]
finish saving first 100 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, all


----------------
record 150 mrr: 0.04444896113029883, hits: [0.08, 0.11333333333333333, 0.14]
finish saving first 150 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, all


----------------
record 200 mrr: 0.037434034463509826, hits: [0.065, 0.095, 0.12]
finish saving first 200 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, all


----------------
record 250 mrr: 0.037785440779773326, hits: [0.068, 0.104, 0.128]
finish saving first 250 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, all


...

----------------
record 600 mrr: 0.0649310597041704, hits: [0.09833333333333333, 0.13, 0.16666666666666666]
finish saving first 600 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, all


----------------
record 650 mrr: 0.06772244727131814, hits: [0.10153846153846154, 0.13692307692307693, 0.17692307692307693]
finish saving first 650 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, all


----------------
record 700 mrr: 0.06368856016769397, hits: [0.09428571428571429, 0.12857142857142856, 0.17285714285714285]
finish saving first 700 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, all


----------------
record 750 mrr: 0.06327358131358077, hits: [0.09466666666666666, 0.128, 0.17066666666666666]
finish saving first 750 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, all


----------------
record 800 mrr: 0.060623084696110274, hits: [0.09, 0.1225, 0.165]
finish saving first 800 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, all


In [ ]:
res_search = pd.read_csv(path+'result/task2/task2_%s_%s_%s.csv'%(qe_mode, include_type, entity))
ranks = res_search.to_dict('record')
start_idx = len(ranks)
start_idx

1040

In [ ]:
res = data[['subjectid', 'objectid', 'predicate']].reset_index().copy()
res_search = pd.DataFrame(ranks)
res = pd.concat([res, res_search], axis=1)


res.to_csv(path+'result/task2/task2_%s_%s_%s.csv'%(qe_mode, include_type, entity), index=False)
res.head()

,index,subjectid,objectid,predicate,idx,rank,dist,list
0,0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",contains,0,23,0.304907,"[poi_way/374898542, poi_way/297756530, line_ro..."
1,1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",contains,1,5,0.300120,"[poi_way/374898542, poi_way/297756530, line_ro..."
2,2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",contains,2,5,0.318047,"[poi_way/297756530, line_poi_way/345753995, li..."
3,3,poi_way/1134399613,poi_way/1134399612,contains,3,5,0.287631,"[poi_way/1134399612, poi_way/1134399600, poi_w..."
4,4,parcel_70920121012,poi_way/1134399612,contains,4,2370,0.320723,"[poi_way/1134399612, poi_way/1134399600, poi_w..."


In [ ]:
res.columns, res.shape

(Index(['index', 'subjectid', 'objectid', 'predicate', 'idx', 'rank', 'dist',
        'list'],
       dtype='object'),
 (1040, 8))

In [ ]:
k_val = [5, 10, 20]

mrr = (1./res['rank'].values).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank'].values<=k).mean())

mrr, hits

(0.06547299787319172,
 [0.09134615384615384, 0.11730769230769231, 0.15576923076923077])

### With subject type

In [ ]:

include_type = 't'
entity = 'subject'

In [ ]:
import time
# llm_abbr = 'none'
qe_mode = 'none'

k_val = [5, 10, 20]

ranks = []
for idx, row in data.iloc[:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    question = 'Retrieve a ' + row['geom_type_subject'] +  ' which ' + text


    subjects = db.similarity_search_with_score(question, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'subjectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['subjectid'][(intersections['objectid']==row['objectid'])&
                                            (intersections['predicate']==row['predicate'])&
                                              (intersections['geom_type_subject']==row['geom_type_subject'])]) - set([row['subjectid']])

    dists = dists[~(dists['subjectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['subjectid']==row['subjectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['subjectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/task2_%s_%s_%s.csv'%(qe_mode, include_type, entity), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

----------------
record 50 mrr: 0.13684944282570113, hits: [0.22, 0.32, 0.32]
finish saving first 50 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 100 mrr: 0.11786473011272579, hits: [0.21, 0.32, 0.33]
finish saving first 100 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 150 mrr: 0.09295826277370603, hits: [0.16666666666666666, 0.26, 0.29333333333333333]
finish saving first 150 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 200 mrr: 0.0989590953728618, hits: [0.165, 0.25, 0.28]
finish saving first 200 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 250 mrr: 0.10837283140624232, hits: [0.18, 0.252, 0.28]
finish saving first 250 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


...

----------------
record 800 mrr: 0.1390618153122341, hits: [0.21375, 0.265, 0.29625]
finish saving first 800 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 850 mrr: 0.14363562105704314, hits: [0.21529411764705883, 0.2635294117647059, 0.2964705882352941]
finish saving first 850 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 900 mrr: 0.13648205115717416, hits: [0.20333333333333334, 0.25222222222222224, 0.2833333333333333]
finish saving first 900 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 950 mrr: 0.13233136300437062, hits: [0.1957894736842105, 0.24421052631578946, 0.27473684210526317]
finish saving first 950 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 1000 mrr: 0.1340080715847811, hits: [0.195, 0.243, 0.275]
finish saving first 1000 searching with none QE, using large as embedding model, gpt4 as llm.with geom_type, t


In [ ]:
res = data[['subjectid', 'objectid', 'predicate']].reset_index().copy()
res_search = pd.DataFrame(ranks)
res = pd.concat([res, res_search], axis=1)


res.to_csv(path+'result/task2/task2_%s_%s_%s.csv'%(qe_mode, include_type, entity), index=False)
res.head()

,index,subjectid,objectid,predicate,idx,rank,dist,list
0,0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",contains,0,3,0.204638,"[road_(5657142535, 53496379), road_(53496379, ..."
1,1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",contains,1,1,0.195867,"[road_(53496379, 5656899646), road_(5657142535..."
2,2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",contains,2,1,0.210956,"[road_(8997363242, 2634716954), road_(26347169..."
3,3,poi_way/1134399613,poi_way/1134399612,contains,3,9,0.212741,"[poi_way/1134399600, poi_way/1134399612, poi_w..."
4,4,parcel_70920121012,poi_way/1134399612,contains,4,311,0.233061,"[parcel_60905452335, parcel_60905452406, parce..."


In [ ]:
res.columns

Index(['index', 'subjectid', 'objectid', 'predicate', 'idx', 'rank', 'dist',
       'list'],
      dtype='object')

In [ ]:
k_val = [5, 10, 20]

mrr = (1./res['rank']).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank'].values<=k).mean())

mrr, hits

(0.15157819074830536, [0.2125, 0.25961538461538464, 0.2903846153846154])

## Geometry generation

### Zero-shot

#### Plain query

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
import langchain
import time

# llm_abbr = 'gpt3_5'

# include_type = 'f'
k_val = [5, 10, 20]

qe_mode = 'q2d_zs'

SYSTEM_PROMPT = """You will receive a spatial query containing the spatial relationship PREDICATE with the reference object B (WKT(B)), and
the desired geometry type of the subject to be retrieved.
Your task is to generate a plausible geometry A in WKT format that could be the result of the query.
The PREDICATE is defined according to the DE-9IM and Open Geospatial Consortium standards.

- Provide only the WKT(A) as the output, and nothing else.
- MAKE SURE the WKT(A) represents a valid Point, LineString, or Polygon as required by the query.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
{query}.
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)


preds = []
for idx, row in data[:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    query = 'Retrieve a ' + row['geom_type_subject'] + ' which ' + text


    pred = chain.invoke({'query':query})

    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/geom_task2_%s.csv'%( qe_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, qe_mode, model_abbr))
        time.sleep(5)

----------------
record 50 predict: LINESTRING (-89.4411 43.0818, -89.441 43.0818, -89.441 43.0817), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of q2d_zs, using gpt4.
----------------
record 100 predict: LINESTRING (-89.4865 43.0622, -89.485 43.0655), actual: ('LineString', 'touches', 'Polygon')
finish saving first 100 predictions of q2d_zs, using gpt4.
----------------
record 150 predict: LINESTRING (-90 43, -89 43), actual: ('LineString', 'contains', 'Point')
finish saving first 150 predictions of q2d_zs, using gpt4.
----------------
record 200 predict: LINESTRING (-89.3985 43.0736, -89.3975 43.074), actual: ('LineString', 'crosses', 'LineString')
finish saving first 200 predictions of q2d_zs, using gpt4.
----------------
record 250 predict: POINT (-89.3244 43.0828), actual: ('Point', 'equals', 'Point')
finish saving first 250 predictions of q2d_zs, using gpt4.
----------------
record 300 predict: POINT(-89.4724 43.0623), actual: ('Point', 'to

In [ ]:
qe_mode = 'q2d_zs'
res_llm = pd.read_csv(path+'/result/task2/geom_task2_%s.csv'%( qe_mode))
preds = res_llm.to_dict('record')
start_idx = len(preds)
start_idx

1000

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
import langchain
import time

# llm_abbr = 'gpt3_5'

# include_type = 'f'
k_val = [5, 10, 20]

qe_mode = 'q2d_zs'

SYSTEM_PROMPT = """You will receive a spatial query containing the spatial relationship PREDICATE with the reference object B (WKT(B)), and
the desired geometry type of the subject to be retrieved.
Your task is to generate a plausible geometry A in WKT format that could be the result of the query.
The PREDICATE is defined according to the DE-9IM and Open Geospatial Consortium standards.

- Provide only the WKT(A) as the output, and nothing else.
- MAKE SURE the WKT(A) represents a valid Point, LineString, or Polygon as required by the query.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
{query}.
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)


# preds = []
for idx, row in data[start_idx:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    query = 'Retrieve a ' + row['geom_type_subject'] + ' which ' + text


    pred = chain.invoke({'query':query})

    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/geom_task2_%s.csv'%( qe_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, qe_mode, model_abbr))
        time.sleep(5)

In [ ]:
res_llm = pd.DataFrame(preds)
print(res_llm.shape)
res_llm.to_csv(path+'/result/task2/geom_task2_%s.csv'%( qe_mode), index=False)

(1040, 3)


In [ ]:
path+'result/task2/geom_task2_%s.csv'%( qe_mode)

'/content/drive/MyDrive/dataset/madison/result/task2/geom_task2_q2d_fs_neg.csv'

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
import langchain
import time

# llm_abbr = 'gpt3_5'

# include_type = 'f'
k_val = [5, 10, 20]

qe_mode = 'q2d_zs'

SYSTEM_PROMPT = """You will receive a spatial query containing the spatial relationship PREDICATE with the reference object B (WKT(B)), and
the desired geometry type of the subject to be retrieved.
Your task is to generate a plausible geometry A in WKT format that could be the result of the query.
The PREDICATE is defined according to the DE-9IM and Open Geospatial Consortium standards.

- Provide only the WKT(A) as the output, and nothing else.
- MAKE SURE the WKT(A) represents a valid Point, LineString, or Polygon as required by the query.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
{query}.
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)


ranks = []
for idx, row in data[:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    query = 'Retrieve a ' + row['geom_type_subject'] + ' which ' + text


    pred = chain.invoke({'query':query})

    query_expanded = query + '. For example, ' + pred['text']


    subjects = db.similarity_search_with_score(query_expanded, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'subjectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['subjectid'][(intersections['objectid']==row['objectid'])&
                                            (intersections['predicate']==row['predicate'])]) - set([row['subjectid']])

    dists = dists[~(dists['subjectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['subjectid']==row['subjectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'], 'qe': pred['text'],
                  'list': dists['subjectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/new_task2_%s_%s_%s.csv'%(embed_abbr, llm_abbr, qe_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.'%(
            idx+1, qe_mode, embed_abbr, llm_abbr))
        time.sleep(5)


----------------
record 50 mrr: 0.12053666629937718, hits: [0.2, 0.28, 0.38]
finish saving first 50 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.


----------------
record 100 mrr: 0.12907559737966806, hits: [0.2, 0.27, 0.32]
finish saving first 100 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.


KeyboardInterrupt: 

In [ ]:
query_expanded

In [ ]:
res_search.head()

In [ ]:
res = data[['subjectid', 'objectid', 'predicate']].reset_index().copy()
res_search = pd.DataFrame(ranks)
res = pd.concat([res, res_search], axis=1)


res.to_csv(path+'result/task2/new_task2_%s_%s_%s.csv'%(qe_mode, include_type, entity), index=False)
res.head()

,index,subjectid,objectid,predicate,idx,rank,dist,qe,list
0,0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",contains,0,4,0.238025,"LINESTRING (-89.3385 43.0922, -89.3388 43.0922)","[road_(5657142535, 53496379), road_(53496379, ..."
1,1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",contains,1,1,0.230326,"LINESTRING (-89.3385 43.0922, -89.3388 43.0922)","[road_(53496379, 5656899646), road_(5657142535..."
2,2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",contains,2,2,0.244149,"LINESTRING (-89.3554 43.1358, -89.3554 43.1354)","[road_(2634716954, 8997363243), road_(89973632..."
3,3,poi_way/1134399613,poi_way/1134399612,contains,3,20,0.245151,"LINESTRING (-89.4524 43.0715, -89.4521 43.0715...","[poi_way/1134399612, poi_way/1134399600, poi_w..."
4,4,parcel_70920121012,poi_way/1134399612,contains,4,196,0.239772,"POLYGON ((-89.4525 43.0716, -89.4520 43.0716, ...","[parcel_60905452335, parcel_60905452406, polyg..."


In [ ]:
k_val = [5, 10, 20]

mrr = (1./res['rank']).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank'].values<=k).mean())

mrr, hits

(0.1791533032800641,
 [0.24423076923076922, 0.2865384615384615, 0.33942307692307694])

#### include check

In [ ]:
qe_mode = 'q2d_zs_check'
res_llm = pd.read_csv(path+'/result/task2/geom_task2_%s.csv'%( qe_mode))
preds = res_llm.to_dict('record')
start_idx = len(preds)
start_idx

1000

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
import langchain
import time

# llm_abbr = 'gpt3_5'

include_type = 'f'
entity = 'subject'
k_val = [5, 10, 20]

qe_mode = 'q2d_zs_check'

SYSTEM_PROMPT = """You will receive a spatial query containing the spatial relationship PREDICATE with the reference object B (WKT(B)), and
the desired geometry type of the subject to be retrieved.
Your task is to generate a plausible geometry A in WKT format that could be the result of the query.
The PREDICATE is defined according to the DE-9IM and Open Geospatial Consortium standards.

- Provide only the WKT(A) as the output, and nothing else.
- MAKE SURE the WKT(A) represents a valid Point, LineString, or Polygon as required by the query.
- MAKE SURE the A exhibits the spatial relationship PREDICATE with B based on the WKT(A).
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
{query}.
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)

# preds = []
for idx, row in data[start_idx:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    query = 'Retrieve a ' + row['geom_type_subject'] + ' which ' + text


    pred = chain.invoke({'query':query})

    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'/result/temp/geom_task2_%s.csv'%( qe_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, qe_mode, model_abbr))
        time.sleep(5)


In [ ]:
res_llm = pd.DataFrame(preds)
print(res_llm.shape[0])
res_llm.to_csv(path+'/result/task2/geom_task2_%s.csv'%( qe_mode), index=False)

1040


In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
import langchain
import time

# llm_abbr = 'gpt3_5'

include_type = 'f'
k_val = [5, 10, 20]

qe_mode = 'q2d_zs_check'

SYSTEM_PROMPT = """You will receive a spatial query containing the spatial relationship PREDICATE with the reference object B (WKT(B)), and
the desired geometry type of the subject to be retrieved.
Your task is to generate a plausible geometry A in WKT format that could be the result of the query.
The PREDICATE is defined according to the DE-9IM and Open Geospatial Consortium standards.

- Provide only the WKT(A) as the output, and nothing else.
- MAKE SURE the WKT(A) represents a valid Point, LineString, or Polygon as required by the query.
- MAKE SURE the A exhibits the spatial relationship PREDICATE with B based on the WKT(A).
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
{query}.
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)
preds = []

ranks = []
for idx, row in data[:5].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    query = 'Retrieve a ' + row['geom_type_subject'] + ' which ' + text


    pred = chain.invoke({'query':query})

    query_expanded = query + '. For example, ' + pred['text']


    subjects = db.similarity_search_with_score(query_expanded, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'subjectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['subjectid'][(intersections['objectid']==row['objectid'])&
                                            (intersections['predicate']==row['predicate'])]) - set([row['subjectid']])

    dists = dists[~(dists['subjectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['subjectid']==row['subjectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'], 'qe': pred['text'],
                  'list': dists['subjectid'].iloc[:10].values.tolist()})

    if (idx+1)%5 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/new_task2_%s_%s_%s.csv'%(embed_abbr, llm_abbr, qe_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.'%(
            idx+1, qe_mode, embed_abbr, llm_abbr))
        time.sleep(5)


----------------
record 5 mrr: 0.3610204081632653, hits: [0.6, 0.6, 0.8]
finish saving first 5 searching with q2d_zs_check QE, using large as embedding model, gpt4 as llm.


In [ ]:
qe_mode

'q2d_zs_check'

In [ ]:
res = data[['subjectid', 'objectid', 'predicate']].reset_index().copy()
res_search = pd.DataFrame(ranks)
res = pd.concat([res, res_search], axis=1)


res.to_csv(path+'result/task2/task2_%s_%s_%s.csv'%(qe_mode, include_type, entity), index=False)
res.head()

,index,subjectid,objectid,predicate,idx,rank,dist,qe,list
0,0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",contains,0,110,0.262709,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922...","[poi_way/278291738, road_(5657142535, 53496379..."
1,1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",contains,1,3,0.253677,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922...","[poi_way/278291738, road_(5657142535, 53496379..."
2,2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",contains,2,2,0.244149,"LINESTRING (-89.3554 43.1358, -89.3554 43.1354)","[road_(2634716954, 8997363243), road_(89973632..."
3,3,poi_way/1134399613,poi_way/1134399612,contains,3,20,0.245151,"LINESTRING (-89.4524 43.0715, -89.4521 43.0715...","[poi_way/1134399612, poi_way/1134399600, poi_w..."
4,4,parcel_70920121012,poi_way/1134399612,contains,4,131,0.238795,"POLYGON((-89.4525 43.0716, -89.4520 43.0716, -...","[parcel_60905452335, parcel_60905452406, polyg..."


In [ ]:
k_val = [5, 10, 20]

mrr = (1./res['rank'].values).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank'].values<=k).mean())

mrr, hits

(0.17558656172058862, [0.24326923076923077, 0.2875, 0.3317307692307692])

### Few-shot

### Positive examples only

In [ ]:
# relations_example = relations_example[relations_example['idx']==250]

In [ ]:
relations_example['n_tokens_subject'].sum() + relations_example['n_tokens'].sum()

105755

In [ ]:
include_type = 't'
entity = 'subject'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

qe_mode = 'q2d_fs'


prefix = """
You will receive a spatial query containing the spatial relationship PREDICATE with the reference object B (WKT(B)), and
the desired geometry type of the subject to be retrieved.
Your task is to generate a plausible geometry A in WKT format that could be the result of the query.
The PREDICATE is defined according to the DE-9IM and Open Geospatial Consortium standards.

- Provide only the WKT(A) as the output, and nothing else.
- MAKE SURE the WKT(A) represents a valid Point, LineString, or Polygon.
- MAKE SURE the A exhibits the spatial relationship PREDICATE with B based on the WKT(A).
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.


[EXAMPLES]
"""

suffix = """
[END EXAMPLES]

Query: {query}
Desired ouatput:
"""

def get_examples(row):
    np.random.seed()
    predicate = row['predicate']
    geom_type = row['geom_type']
    geom_type_subject = row['geom_type_subject']
    # create the example set
    df = relations_example[(relations_example['predicate']==predicate) &
                            (relations_example['geom_type']==geom_type)&
                           (relations_example['geom_type_subject']==geom_type_subject)
                           ]
    df['text'] = df['predicate'] + ' ' + df['geometry']
    df.loc[df['predicate']=='within', 'text'] = 'is ' + df.loc[df['predicate']=='within', 'text']
    examples = (df[['text', 'geom_type_subject', 'geometry_subject']].sample(4).rename(
        columns={'geometry_subject': 'WKT_A',
                 'geom_type_subject': 'type_A'}).to_dict('records')
    )

    return examples

example_prompt = PromptTemplate(input_variables=["WKT_A", "text", "type_A"],
                                    template="Query: Retrieve a {type_A} which {text}\nDesired output: {WKT_A}\n"
    )


preds = []
for idx, row in data[:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    query = 'Retrieve a ' + row['geom_type_subject'] + ' which ' + text


    pred = chain.invoke({'query':query})

    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/geom_task2_%s.csv'%( qe_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, qe_mode, model_abbr))
        time.sleep(5)

In [ ]:
res_llm = pd.DataFrame(preds)
print(res_llm.shape[0])
res_llm.to_csv(path+'result/task2/geom_task2_%s.csv'%( qe_mode), index=False)

1040


#### Positive + Negative

In [ ]:
qe_mode = 'q2d_fs_neg'
res_llm = pd.read_csv(path+'/result/task2/geom_task2_%s.csv'%( qe_mode))
preds = res_llm.to_dict('record')
start_idx = len(preds)
start_idx

1000

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

qe_mode = 'q2d_fs_neg'


prefix = """
You will receive a spatial query containing the spatial relationship PREDICATE with the reference object B (WKT(B)), and
the desired geometry type of the subject to be retrieved.
Your task is to generate a plausible geometry A in WKT format that could be the result of the query.
The PREDICATE is defined according to the DE-9IM and Open Geospatial Consortium standards.

- Provide only the WKT(A) as a good output, and nothing else.
- MAKE SURE the WKT(A) represents a valid Point, LineString, or Polygon.
- MAKE SURE the A exhibits the spatial relationship PREDICATE with B based on the WKT(A).
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.


[EXAMPLES]
"""

suffix = """
[END EXAMPLES]

Query: {query}
Desired output:
"""

def get_examples(row):
    np.random.seed()
    predicate = row['predicate']
    geom_type = row['geom_type']
    geom_type_subject = row['geom_type_subject']
    # create the example set
    df = relations_example[(relations_example['predicate']==predicate) &
                            (relations_example['geom_type']==geom_type)&
                           (relations_example['geom_type_subject']==geom_type_subject)]

    df['text'] = df['predicate'] + ' ' + df['geometry']
    df = pd.merge(df, (relations[['subjectid', 'predicate',
                                 'objectid', 'geometry_subject', 'geom_type_subject']]
                  .rename(columns={'predicate': 'bad_predicate',
                                   'geometry_subject': 'WKT_bad',
                                   })),
                  on=['objectid', 'geom_type_subject'], how='inner')
    df = df[df['predicate']!=df['bad_predicate']].drop_duplicates(subset=['objectid', 'predicate'])
    df.loc[df['predicate']=='within', 'text'] = 'is ' + df.loc[df['predicate']=='within', 'text']
    examples = (df[['text', 'geometry_subject', "geom_type_subject", 'WKT_bad']].sample(4).rename(
        columns={'geometry_subject': 'WKT_good',
                 "geom_type_subject": "type"}).to_dict('records')
    )

    return examples

example_prompt = PromptTemplate(input_variables=["type", "WKT_good", "text", "WKT_bad"],
                                    template="Query: Retrieve a {type} which {text}\nGood response: {WKT_good}\nBad response: {WKT_bad}"
    )


# preds = []
for idx, row in data[start_idx:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    query = 'Retrieve a ' + row['geom_type_subject'] + ' which ' + text


    pred = chain.invoke({'query':query})

    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/geom_task2_%s.csv'%( qe_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, qe_mode, model_abbr))
        time.sleep(5)

In [ ]:
res_llm = pd.DataFrame(preds)
print(res_llm.shape[0])
res_llm.to_csv(path+'result/task2/geom_task2_%s.csv'%( qe_mode), index=False)

1040


## Retrieval

### One query + One Geometry

In [ ]:
qe_mode = 'q2d_zs'
res_llm = pd.read_csv(path+'/result/task2/geom_task2_%s.csv'%( qe_mode))
preds = res_llm.to_dict('record')

In [ ]:
import time
# llm_abbr = 'none'
# qe_mode = 'none'
repeat = 1

k_val = [5, 10, 20]

ranks = []
for idx, row in data.iloc[:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    question = 'Retrieve a ' + row['geom_type_subject'] +  ' which ' + text +'.'

    query = question + ' For example, ' + preds[idx]['text'] +'.'
    # print(idx, query)
    subjects = db.similarity_search_with_score(query, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'subjectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['subjectid'][(intersections['objectid']==row['objectid'])&
                                            (intersections['predicate']==row['predicate'])
                                            &(intersections['geom_type_subject']==row['geom_type_subject'])]) - set([row['subjectid']])

    dists = dists[~(dists['subjectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['subjectid']==row['subjectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['subjectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/new_task2_res_%s_%d.csv'%(qe_mode, repeat), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

----------------
record 50 mrr: 0.120711413634192, hits: [0.16, 0.2, 0.32]
finish saving first 50 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 100 mrr: 0.1267496770258093, hits: [0.17, 0.23, 0.3]
finish saving first 100 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 150 mrr: 0.10640694610229377, hits: [0.14666666666666667, 0.19333333333333333, 0.2733333333333333]
finish saving first 150 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 200 mrr: 0.11245870648618603, hits: [0.15, 0.21, 0.275]
finish saving first 200 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 250 mrr: 0.1201483675941893, hits: [0.172, 0.224, 0.276]
finish saving first 250 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


...

----------------
record 800 mrr: 0.16849648859411687, hits: [0.23625, 0.2825, 0.3325]
finish saving first 800 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 850 mrr: 0.17335332021523286, hits: [0.2388235294117647, 0.28352941176470586, 0.3305882352941176]
finish saving first 850 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 900 mrr: 0.16557430356359001, hits: [0.2288888888888889, 0.27111111111111114, 0.32]
finish saving first 900 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 950 mrr: 0.16079633072913066, hits: [0.22, 0.26105263157894737, 0.31157894736842107]
finish saving first 950 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 1000 mrr: 0.1605256709589573, hits: [0.22, 0.261, 0.313]
finish saving first 1000 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


In [ ]:
res_search = pd.DataFrame(ranks)
print(res_search.shape)
res_search.to_csv(path+'result/task2/new_task2_res_%s_%d.csv'%(qe_mode, repeat))

(1040, 4)


In [ ]:
k_val = [5, 10, 20]
res = pd.DataFrame(ranks)
mrr = (1./res['rank']).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank'].values<=k).mean())

mrr, hits

(0.17970460884413905,
 [0.23846153846153847, 0.2778846153846154, 0.3278846153846154])

### One query + three geoms

In [ ]:
qe_mode = 'q2d_zs'
res_llm = pd.read_csv(path+'/result/task2/geom_task2_%s.csv'%( qe_mode))
preds2 = res_llm.to_dict('record')

res_llm = pd.read_csv(path+'/result/task2/geom_0_task2_%s.csv'%( qe_mode))
preds0 = res_llm.to_dict('record')

res_llm = pd.read_csv(path+'/result/task2/geom_1_task2_%s.csv'%( qe_mode))
preds1 = res_llm.to_dict('record')

In [ ]:
import time
# llm_abbr = 'none'
# qe_mode = 'none'
repeat = 1

k_val = [5, 10, 20]

ranks = []
for idx, row in data.iloc[:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    question = 'Retrieve a ' + row['geom_type_subject'] +  ' which ' + text +'.'

    query = question + ' For example, ' + preds0[idx]['text'] +',' + preds1[idx]['text'] + ',' + preds2[idx]['text']
    # print(idx, query)
    subjects = db.similarity_search_with_score(question, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'subjectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['subjectid'][(intersections['objectid']==row['objectid'])&
                                            (intersections['predicate']==row['predicate'])
                                            &(intersections['geom_type_subject']==row['geom_type_subject'])]) - set([row['subjectid']])

    dists = dists[~(dists['subjectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['subjectid']==row['subjectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['subjectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/new_task2_res_%s_d3.csv'%(qe_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

----------------
record 50 mrr: 0.13265004021600904, hits: [0.24, 0.34, 0.36]
finish saving first 50 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 100 mrr: 0.1186800827559377, hits: [0.21, 0.35, 0.36]
finish saving first 100 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 150 mrr: 0.09972558055725667, hits: [0.16, 0.26666666666666666, 0.31333333333333335]
finish saving first 150 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 200 mrr: 0.10847130056285348, hits: [0.165, 0.26, 0.305]
finish saving first 200 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 250 mrr: 0.11599377077131967, hits: [0.18, 0.264, 0.3]
finish saving first 250 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


...

----------------
record 800 mrr: 0.15750090238306214, hits: [0.235, 0.28625, 0.32625]
finish saving first 800 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 850 mrr: 0.16160879136325107, hits: [0.23529411764705882, 0.2847058823529412, 0.3235294117647059]
finish saving first 850 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 900 mrr: 0.15386124348589567, hits: [0.22444444444444445, 0.2722222222222222, 0.31222222222222223]
finish saving first 900 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 950 mrr: 0.14882621303506366, hits: [0.21578947368421053, 0.2631578947368421, 0.3021052631578947]
finish saving first 950 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 1000 mrr: 0.14975176259089176, hits: [0.214, 0.261, 0.302]
finish saving first 1000 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


In [ ]:
res_search = pd.DataFrame(ranks)
print(res_search.shape)
res_search.to_csv(path+'result/task2/new_task2_res_%s_d3.csv'%(qe_mode))

(1040, 4)


In [ ]:
k_val = [5, 10, 20]
res = pd.DataFrame(ranks)
mrr = (1./res['rank']).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank'].values<=k).mean())

mrr, hits

(0.1688592076544702,
 [0.23173076923076924, 0.2798076923076923, 0.32019230769230766])

### Three query + one geometry

In [ ]:
import time
# llm_abbr = 'none'
# qe_mode = 'none'
repeat = 3

k_val = [5, 10, 20]

ranks = []
for idx, row in data.iloc[:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    question = 'Retrieve a ' + row['geom_type_subject'] +  ' which ' + text +'.'

    query = question + question + question + ' For example, ' + preds[idx]['text'] +'.'
    # print(idx, query)
    subjects = db.similarity_search_with_score(question, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'subjectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['subjectid'][(intersections['objectid']==row['objectid'])&
                                            (intersections['predicate']==row['predicate'])&
                                              (intersections['geom_type_subject']==row['geom_type_subject'])]) - set([row['subjectid']])

    dists = dists[~(dists['subjectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['subjectid']==row['subjectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['subjectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/new_task2_res_%s_%d.csv'%(qe_mode, repeat), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

----------------
record 50 mrr: 0.13265030254874827, hits: [0.24, 0.34, 0.36]
finish saving first 50 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 100 mrr: 0.11868017609556718, hits: [0.21, 0.35, 0.36]
finish saving first 100 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 150 mrr: 0.10044005748103825, hits: [0.16666666666666666, 0.26666666666666666, 0.31333333333333335]
finish saving first 150 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 200 mrr: 0.10900715825568968, hits: [0.17, 0.26, 0.305]
finish saving first 200 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 250 mrr: 0.11642245178885659, hits: [0.184, 0.264, 0.3]
finish saving first 250 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


...

----------------
record 800 mrr: 0.15763192886265492, hits: [0.23625, 0.28625, 0.32625]
finish saving first 800 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 850 mrr: 0.161734911523316, hits: [0.23647058823529413, 0.2847058823529412, 0.3235294117647059]
finish saving first 850 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 900 mrr: 0.1539798719934474, hits: [0.22555555555555556, 0.2722222222222222, 0.31222222222222223]
finish saving first 900 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 950 mrr: 0.14893859793695474, hits: [0.2168421052631579, 0.2631578947368421, 0.3021052631578947]
finish saving first 950 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


----------------
record 1000 mrr: 0.14985853044854688, hits: [0.215, 0.261, 0.302]
finish saving first 1000 searching with q2d_zs QE, using large as embedding model, gpt4 as llm.with geom_type, t


In [ ]:
res_search = pd.DataFrame(ranks)
print(res_search.shape)
res_search.to_csv(path+'result/task2/new_task2_res_%s_%d.csv'%(qe_mode, repeat))

(1040, 4)


In [ ]:
k_val = [5, 10, 20]
res = pd.DataFrame(ranks)
mrr = (1./res['rank']).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank'].values<=k).mean())

mrr, hits

(0.16899239408658673,
 [0.2326923076923077, 0.2798076923076923, 0.32019230769230766])

In [ ]:
qe_mode = 'q2d_zs'
res_llm = pd.read_csv(path+'/result/task2/geom_task2_%s.csv'%( qe_mode))
preds = res_llm.to_dict('record')
len(preds)

1040

In [ ]:
qe_mode = 'q2d_zs'
repeat = 3
res_search = pd.read_csv(path+'result/temp/task2_res_%s_%d.csv'%(qe_mode, repeat))
ranks = res_search.to_dict('record')
start_idx = len(ranks)
start_idx

1000

In [ ]:
idx

1343

In [ ]:
import time
# llm_abbr = 'none'
# qe_mode = 'none'
repeat = 3

k_val = [5, 10, 20]

# ranks = []
for idx, row in data.iloc[start_idx:].iterrows():
    text = row['predicate'] + ' ' + row['geometry']
    text = 'is ' + text if row['predicate']=='within' else text
    question = 'Retrieve a ' + row['geom_type_subject'] +  ' which ' + text +'.'

    query = question + ' For example, ' + preds[idx]['text'] +'.'
    # print(idx, query)
    subjects = db.similarity_search_with_score(question, k=objects.shape[0])
    dists = []
    for subject in subjects:
        dists.append({'subjectid': subject[0].metadata['objectid'], 'dist': subject[1]})
    dists = pd.DataFrame(dists)
    positive = set(intersections['subjectid'][(intersections['objectid']==row['objectid'])&
                                            (intersections['predicate']==row['predicate'])]) - set([row['subjectid']])

    dists = dists[~(dists['subjectid'].isin(positive))].sort_values(by='dist')
    dists['rank'] = np.arange(dists.shape[0])+1
    rank = dists[dists['subjectid']==row['subjectid']].iloc[0]
    ranks.append({'idx': idx, 'rank': rank['rank'], 'dist': rank['dist'],
                  'list': dists['subjectid'].iloc[:10].values.tolist()})

    if (idx+1)%50 == 0:
        res_search = pd.DataFrame(ranks)
        mrr = (1./res_search['rank'].values).mean()
        hits = []
        for k in k_val:
            hits.append(np.array(res_search['rank'].values<=k).mean())

        res_search.to_csv(path+'result/temp/task2_res_%s_%d.csv'%(qe_mode, repeat), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'mrr: %s, hits: %s'%(mrr, hits))
        print('finish saving first %d searching with %s QE, using %s as embedding model, %s as llm.with geom_type, % s'%(
            idx+1, qe_mode, embed_abbr, llm_abbr, include_type))
        time.sleep(5)

In [ ]:
res_search = pd.DataFrame(ranks)
print(res_search.shape)
res_search.to_csv(path+'result/task2/task2_res_%s_%d.csv'%(qe_mode, repeat))

(1040, 4)


In [ ]:
k_val = [5, 10, 20]
res = pd.DataFrame(ranks)
mrr = (1./res['rank']).mean()
hits = []
for k in k_val:
    hits.append(np.array(res['rank'].values<=k).mean())

mrr, hits

(0.16895709372846893,
 [0.23173076923076924, 0.2798076923076923, 0.32019230769230766])